Q-러닝(Q-learning)

# 개요

In [ ]:
# 금융 분야에서 강화 학습을 사용하여 최적의 투자 전략이나 포트폴리오 관리를 학습하는 문제는 많이 연구되고 있습니다. 
# 여기서는 간단한 강화 학습 알고리즘을 사용해 주식 거래 시나리오를 시뮬레이션하는 예시를 파이썬으로 구현해보겠습니다.

In [ ]:
#  Q-러닝(Q-learning) 알고리즘을 사용하여 
# 주식 거래 에이전트가 매수(buy), 매도(sell) 또는 **유지(hold)**와 같은 행동을 선택하며, 
# 포트폴리오의 수익을 극대화하는 전략을 학습하는 방식입니다.

In [ ]:
# 환경 설정
# 에이전트는 주식의 가격 변동에 따라 행동을 선택합니다.
# 상태는 과거 몇 일간의 주가 데이터를 기반으로 하며, 에이전트는 해당 상태에서 행동을 결정합니다.
# 행동은 매수, 매도, 유지의 세 가지 중 하나입니다.
# 목표는 최종 수익을 극대화하는 것입니다.

# 실습

## 라이브러리

In [1]:
import numpy as np
import pandas as pd
import random

## 주식 가격 데이터 생성

In [ ]:
# 랜덤으로 주식 가격을 시뮬레이션합니다. 
# 실제 주식 데이터를 사용할 경우, CSV 파일을 읽어서 처리할 수도 있습니다.

In [2]:
# 간단한 주식 가격 데이터를 생성 (랜덤)
np.random.seed(42)
stock_prices = np.random.normal(100, 10, 100)  # 주식 가격 시뮬레이션 (100일간)

In [3]:
stock_prices 

array([104.96714153,  98.61735699, 106.47688538, 115.23029856,
        97.65846625,  97.65863043, 115.79212816, 107.67434729,
        95.30525614, 105.42560044,  95.36582307,  95.34270246,
       102.41962272,  80.86719755,  82.75082167,  94.37712471,
        89.8716888 , 103.14247333,  90.91975924,  85.87696299,
       114.65648769,  97.742237  , 100.67528205,  85.75251814,
        94.55617275, 101.1092259 ,  88.49006423, 103.75698018,
        93.9936131 ,  97.0830625 ,  93.98293388, 118.52278185,
        99.86502775,  89.42289071, 108.22544912,  87.7915635 ,
       102.08863595,  80.40329876,  86.71813951, 101.96861236,
       107.3846658 , 101.71368281,  98.84351718,  96.98896304,
        85.2147801 ,  92.80155792,  95.39361229, 110.57122226,
       103.4361829 ,  82.36959845, 103.24083969,  96.1491772 ,
        93.23078   , 106.11676289, 110.30999522, 109.31280119,
        91.60782477,  96.90787624, 103.31263431, 109.75545127,
        95.20825762,  98.14341023,  88.93665026,  88.03

 ## Q-러닝 파라미터 설정

In [ ]:
# 감마(γ): 미래의 보상을 얼마나 중요하게 여길지를 결정하는 파라미터로, 장기적인 전략을 고려할 때 감마 값을 높게 설정합니다.
# 감마 값이 높으면, 먼 미래의 보상까지 중요하게 고려하고, 감마 값이 낮으면 즉각적인 보상에 더 큰 가중치를 둡니다.
#예를 들어, 𝛾=0.9일 경우, 에이전트는 미래 보상도 중요하게 여기며 장기적인 전략을 학습하려고 합니다. 
#반면에 𝛾=0.1이면 에이전트는 즉각적인 보상을 중시하게 됩니다.

# 엡실론(ε)
#: 탐험과 활용의 균형을 맞추는 파라미터로, 초기에는 탐험을 많이 하고 점차 활용으로 전환해야 합니다.
# 엡실론은 에이전트가 탐험(Exploration)과 활용(Exploitation) 사이에서 어느 정도로 탐험을 할지 결정하는 파라미터입니다.
# 값의 범위: 0≤𝜖≤1
# 역할:

# 탐험(Exploration)
#: 에이전트가 새로운 행동을 시도하는 것, 즉 Q-테이블에 아직 반영되지 않은 상태와 행동을 탐험하는 것을 말합니다. 


# 최소 탐험(min_epsilon)
# : 학습 후반에도 소량의 탐험을 유지하기 위해 엡실론 값이 감소하는 하한선을 설정합니다.
# 엡실론 값이 줄어들 수 있는 최소한의 값을 지정하는 파라미터입니다.
# 역할: 학습이 끝나기 전에 엡실론 값이 0에 도달하지 않도록 하여, 학습 후반부에도 소량의 탐험이 남아있도록 설정합니다. 
# 이로 인해 새로운 상태나 행동을 탐색할 수 있는 여지가 계속 남게 됩니다.
# 보통 0.01 또는 0.05 정도로 설정하여, 학습이 끝나갈 때도 소량의 탐험이 계속 유지되도록 합니다.

# 학습률(α)
# : 새로운 정보를 기존 정보에 얼마나 반영할지를 결정하는 파라미터로, 적절한 값을 설정해야 빠르고 안정적인 학습이 가능합니다.
# 0.01≤𝛼≤0.3 사이를 선택하여 균형을 맞추는 것이 좋습니다.

# 에피소드 수(episodes)
# 에피소드 수는 강화 학습의 학습 과정에서 에이전트가 환경과 상호작용하며 경험을 축적하는 반복 횟수를 의미합니다. 
# 하나의 에피소드는 초기 상태에서 시작하여 종료 상태에 도달할 때까지의 한 과정입니다.
# : 에이전트가 충분히 학습할 수 있는 기회를 제공하며, 에피소드 수가 많을수록 학습의 완성도가 높아집니다.

In [4]:
# Q-러닝 파라미터 설정
gamma = 0.95  # 감가율
epsilon = 1.0  # 탐험과 활용 사이의 균형
epsilon_decay = 0.995  # 탐험 감소 속도
min_epsilon = 0.01  # 최소 탐험
learning_rate = 0.01  # 학습률
num_episodes = 1000  # 에피소드 수

## 행동 정의

In [6]:
# 행동 정의 (0: 유지, 1: 매수, 2: 매도)
actions = [0, 1, 2]

## Q-테이블 초기화

In [7]:
# Q-테이블 초기화:
# 각 상태에서의 행동에 대한 Q-값을 저장할 테이블입니다. 
# 상태는 시간 단계로 구성되며, 각 시간 단계에서 에이전트는 매수, 매도, 유지 중 하나의 행동을 선택합니다.

In [9]:
# Q-테이블 초기화 (상태 x 행동)
Q_table = np.zeros((len(stock_prices), len(actions)))

# 에이전트의 초기 상태 (주식 보유 여부, 현금, 포트폴리오 가치) 초기금액 : 10000
initial_cash = 10000
initial_stock_held = 0
initial_portfolio_value = initial_cash

## 강화학습 환경 정의

In [10]:

def get_reward(cash, stock_held, portfolio_value, current_price, previous_price):
    # 포트폴리오의 가치가 증가했으면 보상 증가
    new_portfolio_value = cash + stock_held * current_price
    reward = new_portfolio_value - portfolio_value
    return reward, new_portfolio_value


## Q-러닝 알고리즘

In [11]:
# 각 에피소드마다 주어진 가격 데이터에 따라 에이전트가 주식을 사고 팔며, 보상을 최대화하도록 Q-값을 업데이트합니다.
# 에이전트는 주식을 매수(buy), 매도(sell) 또는 **유지(hold)**하는 세 가지 행동 중 하나를 선택합니다.
# 에이전트는 미래의 보상을 고려하며 행동을 선택하고, 각 행동에 따른 포트폴리오 가치의 변화를 학습합니다.

# 탐험과 활용:
# 학습 초기에는 에이전트가 탐험(랜덤 행동)을 많이 하지만, 학습이 진행될수록 최적의 행동을 선택하는 확률이 증가합니다 (탐험-활용 균형).

# 보상 함수:
# 보상은 에이전트의 포트폴리오 가치가 얼마나 증가했는지를 기반으로 합니다. 포트폴리오 가치가 증가하면 긍정적인 보상을 받고, 감소하면 음의 보상을 받습니다.
# Q-값 업데이트:

# 벨만 방정식을 기반으로 Q-값을 업데이트하며, 이 과정에서 학습률과 감가율을 고려합니다.


In [12]:
# Q-러닝 알고리즘
for episode in range(num_episodes):
    cash = initial_cash
    stock_held = initial_stock_held
    portfolio_value = initial_portfolio_value
    state = 0  # 첫 번째 상태부터 시작

    for t in range(len(stock_prices) - 1):
        current_price = stock_prices[state]
        next_price = stock_prices[state + 1]

        # 탐험(Exploration) 또는 활용(Exploitation) 결정
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)  # 랜덤 선택 (탐험)
        else:
            action = np.argmax(Q_table[state])  # 최적 행동 선택 (활용)

        # 행동에 따른 상태 변화
        if action == 1 and cash >= current_price:  # 매수
            stock_held += 1
            cash -= current_price
        elif action == 2 and stock_held > 0:  # 매도
            stock_held -= 1
            cash += current_price

        # 보상 계산
        reward, new_portfolio_value = get_reward(cash, stock_held, portfolio_value, next_price, current_price)

        # Q-값 업데이트 (벨만 방정식)
        next_state = state + 1
        Q_table[state, action] = Q_table[state, action] + learning_rate * (
            reward + gamma * np.max(Q_table[next_state]) - Q_table[state, action]
        )

        # 상태 업데이트
        state = next_state
        portfolio_value = new_portfolio_value

    # 탐험-활용 균형 조절 (epsilon 감소)
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay
# 학습 후의 Q-테이블 확인
print("Q-테이블:")
print(Q_table)

Q-테이블:
[[  5.59764794   2.22591851  32.83946911]
 [  6.76402517  36.72362856   4.32446512]
 [  3.8375748   32.49534441  -0.09167433]
 [ -6.91918691 -14.07953943  17.9697408 ]
 [ 39.80305362   8.06270898   9.15994982]
 [  7.70483387  45.07764635   2.56993649]
 [ -5.80566782 -10.71109234  13.06714072]
 [ -3.51595355  -9.64811491  26.84630094]
 [  4.02845942  33.22907543   3.17340719]
 [ -5.75750503 -11.42536308  29.21849515]
 [ 35.56115875   3.07801044   3.93805448]
 [  3.04193293  41.42653768   1.72663599]
 [-13.84759415 -26.16883527  39.85403976]
 [ 45.46129106   9.36720435  11.01162969]
 [ 12.27926054  50.45686533   9.24733256]
 [  4.8063748    4.38842047  43.44758843]
 [  9.80460219  49.25212163   6.43722994]
 [ -7.65468307 -11.66547676  41.20681524]
 [ 10.35643757   7.00536451  48.25823762]
 [ 23.69458287  55.58940559   9.77138096]
 [-18.35270343 -24.79878496  31.99427013]
 [  2.56715027  39.12470923   2.26255508]
 [ -8.22683746 -17.1212481   42.54869639]
 [ 14.24568538  49.9104664 

## 학습 후 정책 테스트

In [13]:
# 학습이 완료된 후, 학습된 Q-테이블을 사용해 최적의 정책을 실행합니다. 
# 에이전트는 학습한 대로 주식을 사고 팔며 포트폴리오 가치를 극대화하려 합니다.

In [14]:
# 학습 후 최적 정책 테스트
cash = initial_cash
stock_held = initial_stock_held
portfolio_value = initial_portfolio_value
state = 0

for t in range(len(stock_prices) - 1):
    current_price = stock_prices[state]
    action = np.argmax(Q_table[state])  # 최적 행동 선택
    if action == 1 and cash >= current_price:  # 매수
        stock_held += 1
        cash -= current_price
        print(f"날짜 {t}: 매수, 주가: {current_price:.2f}")
    elif action == 2 and stock_held > 0:  # 매도
        stock_held -= 1
        cash += current_price
        print(f"날짜 {t}: 매도, 주가: {current_price:.2f}")
    state += 1

final_portfolio_value = cash + stock_held * stock_prices[-1]
print(f"최종 포트폴리오 가치: {final_portfolio_value:.2f}")


날짜 1: 매수, 주가: 98.62
날짜 2: 매수, 주가: 106.48
날짜 3: 매도, 주가: 115.23
날짜 5: 매수, 주가: 97.66
날짜 6: 매도, 주가: 115.79
날짜 7: 매도, 주가: 107.67
날짜 8: 매수, 주가: 95.31
날짜 9: 매도, 주가: 105.43
날짜 11: 매수, 주가: 95.34
날짜 12: 매도, 주가: 102.42
날짜 14: 매수, 주가: 82.75
날짜 15: 매도, 주가: 94.38
날짜 16: 매수, 주가: 89.87
날짜 17: 매도, 주가: 103.14
날짜 19: 매수, 주가: 85.88
날짜 20: 매도, 주가: 114.66
날짜 21: 매수, 주가: 97.74
날짜 22: 매도, 주가: 100.68
날짜 23: 매수, 주가: 85.75
날짜 25: 매도, 주가: 101.11
날짜 26: 매수, 주가: 88.49
날짜 27: 매도, 주가: 103.76
날짜 28: 매수, 주가: 93.99
날짜 29: 매도, 주가: 97.08
날짜 30: 매수, 주가: 93.98
날짜 31: 매도, 주가: 118.52
날짜 33: 매수, 주가: 89.42
날짜 34: 매도, 주가: 108.23
날짜 35: 매수, 주가: 87.79
날짜 36: 매도, 주가: 102.09
날짜 37: 매수, 주가: 80.40
날짜 38: 매수, 주가: 86.72
날짜 39: 매수, 주가: 101.97
날짜 40: 매도, 주가: 107.38
날짜 41: 매도, 주가: 101.71
날짜 42: 매도, 주가: 98.84
날짜 44: 매수, 주가: 85.21
날짜 45: 매도, 주가: 92.80
날짜 46: 매수, 주가: 95.39
날짜 47: 매도, 주가: 110.57
날짜 49: 매수, 주가: 82.37
날짜 50: 매도, 주가: 103.24
날짜 52: 매수, 주가: 93.23
날짜 53: 매수, 주가: 106.12
날짜 54: 매도, 주가: 110.31
날짜 55: 매도, 주가: 109.31
날짜 57: 매수, 주가: 96.91

In [ ]:
# 결과
# 에이전트는 주어진 주식 가격 데이터를 통해 최적의 거래 전략을 학습합니다.
# 최종적으로, 에이전트의 포트폴리오 가치가 얼마나 증가했는지 확인할 수 있습니다.

# 초기금액 : 10000==> 최종 포트폴리오 가치: 10456.57

# 코드 종합

In [ ]:
import numpy as np
import pandas as pd
import random

# 간단한 주식 가격 데이터를 생성 (랜덤)
np.random.seed(42)
stock_prices = np.random.normal(100, 10, 100)  # 주식 가격 시뮬레이션 (100일간)

# Q-러닝 파라미터 설정
gamma = 0.95  # 감가율
epsilon = 1.0  # 탐험과 활용 사이의 균형
epsilon_decay = 0.995  # 탐험 감소 속도
min_epsilon = 0.01  # 최소 탐험
learning_rate = 0.01  # 학습률
num_episodes = 1000  # 에피소드 수

# 행동 정의 (0: 유지, 1: 매수, 2: 매도)
actions = [0, 1, 2]

# Q-테이블 초기화 (상태 x 행동)
Q_table = np.zeros((len(stock_prices), len(actions)))

# 에이전트의 초기 상태 (주식 보유 여부, 현금, 포트폴리오 가치)
initial_cash = 10000
initial_stock_held = 0
initial_portfolio_value = initial_cash

# 강화학습 환경 정의
def get_reward(cash, stock_held, portfolio_value, current_price, previous_price):
    # 포트폴리오의 가치가 증가했으면 보상 증가
    new_portfolio_value = cash + stock_held * current_price
    reward = new_portfolio_value - portfolio_value
    return reward, new_portfolio_value

# Q-러닝 알고리즘
for episode in range(num_episodes):
    cash = initial_cash
    stock_held = initial_stock_held
    portfolio_value = initial_portfolio_value
    state = 0  # 첫 번째 상태부터 시작

    for t in range(len(stock_prices) - 1):
        current_price = stock_prices[state]
        next_price = stock_prices[state + 1]

        # 탐험(Exploration) 또는 활용(Exploitation) 결정
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)  # 랜덤 선택 (탐험)
        else:
            action = np.argmax(Q_table[state])  # 최적 행동 선택 (활용)

        # 행동에 따른 상태 변화
        if action == 1 and cash >= current_price:  # 매수
            stock_held += 1
            cash -= current_price
        elif action == 2 and stock_held > 0:  # 매도
            stock_held -= 1
            cash += current_price

        # 보상 계산
        reward, new_portfolio_value = get_reward(cash, stock_held, portfolio_value, next_price, current_price)

        # Q-값 업데이트 (벨만 방정식)
        next_state = state + 1
        Q_table[state, action] = Q_table[state, action] + learning_rate * (
            reward + gamma * np.max(Q_table[next_state]) - Q_table[state, action]
        )

        # 상태 업데이트
        state = next_state
        portfolio_value = new_portfolio_value

    # 탐험-활용 균형 조절 (epsilon 감소)
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay

# 학습 후의 Q-테이블 확인
print("Q-테이블:")
print(Q_table)

# 학습 후 최적 정책 테스트
cash = initial_cash
stock_held = initial_stock_held
portfolio_value = initial_portfolio_value
state = 0

for t in range(len(stock_prices) - 1):
    current_price = stock_prices[state]
    action = np.argmax(Q_table[state])  # 최적 행동 선택
    if action == 1 and cash >= current_price:  # 매수
        stock_held += 1
        cash -= current_price
        print(f"날짜 {t}: 매수, 주가: {current_price:.2f}")
    elif action == 2 and stock_held > 0:  # 매도
        stock_held -= 1
        cash += current_price
        print(f"날짜 {t}: 매도, 주가: {current_price:.2f}")
    state += 1

final_portfolio_value = cash + stock_held * stock_prices[-1]
print(f"최종 포트폴리오 가치: {final_portfolio_value:.2f}")
